# Client-Side Code

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model

from PIL import Image

### Inference Model

In [48]:
# Half of the Inference Model Stored on Client Side
class ClientResNet(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(ClientResNet, self).__init__(*args, **kwargs)
        
        # Load ResNet50 pre-trained model without top (fully connected) layers
        resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        resnet_base.trainable = False
        
        # Get the output of the first three convolutional layers
        middle_layer = resnet_base.get_layer('conv3_block4_out')
        self.seq0 = Model(inputs=resnet_base.input, outputs=middle_layer.output)

    # Convert image into right dimensions
    def preprocess_image(self, img):
        img_array = kimage.img_to_array(img)
        expand_img = np.expand_dims(img_array, axis=0)
        return preprocess_input(expand_img)
    
    def add_laplacian_noise(self, emb, noise_scale=0.5):
        laplace_noise = np.random.laplace(0.0, noise_scale, size=emb.shape)
        emb += laplace_noise
        return emb
        
    # Forward pass of the client model
    def predict(self, image_path, laplace_noise=False):
        # Open and process the image
        img = Image.open(image_path)
        img = img.resize((224, 224)).convert("RGB")

        preprocessed_img = self.preprocess_image(img)
        img.close()

        # Feed through ResNet
        emb = self.seq0(preprocessed_img)

        # Apply noise
        if laplace_noise:
            emb = self.add_laplacian_noise(emb)

        return emb

# Load in Model
ClientModel = ClientResNet()

In [49]:
# Perform Inference on image path
image_path = "jeans.png"
client_embeddings = ClientModel.predict(image_path, False)

# Replace: This is a mimic for sending to server
np.save("client_embeddings.npy", client_embeddings)

noise_embeddings = ClientModel.predict(image_path, True)
np.save("noise_embeddings.npy", noise_embeddings)

### Checklist
1) Move Laplace Noise into model ✅
2) Write Siamese Noise
    - Add color classification
    - Add item classification
    - For each entry in the embedding, move 'alpha' closer to the cluster 
3) Add Distance Metric to Test
4) Split ResNet Loading for size reasons
5) Create ReadME of workflow

In [5]:
# See model layers
import tensorflow.keras.applications as keras_applications

model = keras_applications.ResNet50(weights='imagenet', include_top=False)
for layer in model.layers:
    print(layer.name, layer.output_shape)